# Anexos: Diseño e Implementación del Control MPPT

Este apartado recopila la documentación técnica, diagramas de simulación y diseño de hardware desarrollados para la implementación del algoritmo de seguimiento del punto de máxima potencia (MPPT) utilizando la técnica *Perturb & Observe* (P&O) en un microcontrolador ATmega328P.

---

## 1. Dimensionamiento del Convertidor DC-DC (Cuk)
Para la etapa de potencia, se seleccionó una topología Cuk debido a su capacidad de proporcionar una corriente de salida continua y baja ondulación. A continuación se presentan los cálculos matemáticos para la selección de inductores ($L_1, L_2$) y capacitores ($C_1, C_2$).

<div align="center">
  <img src="../assets\images\Valores Cuk_page-0001.jpg" alt="Calculos Matemáticos Convertidor Cuk" width="600"/>
  <p><em>Fig 1. Hoja de cálculo para el dimensionamiento de componentes pasivos considerando una potencia MPPT de 25W y Vout de 12V.</em></p>
</div>

---

## 2. Simulación en MATLAB/Simulink
El sistema fue validado mediante simulación antes de la implementación física. Se modeló el panel solar, la etapa de potencia y el lazo de control.

### 2.1 Topología del Circuito
<div align="center">
  <img src="../assets\images\Topologia.png" alt="Esquematico Simulink" width="850"/>
  <p><em>Fig 2. Diagrama esquemático en Simulink. Se observan los valores calculados: L1=660uH, L2=550uH y el capacitor de acoplo de 220nF.</em></p>
</div>

### 2.2 Bloques de Control
<div align="center">
  <img src="../assets/images/Captura de pantalla 2025-12-11 223157.png" alt="Control Simulink" width="800"/>
  <p><em>Fig 3. Interfaz de simulación con bloques de medición de voltaje y corriente ($V_{pv}, I_{pv}$) y generación de señal PWM para el control del MOSFET.</em></p>
</div>

---

## 3. Lógica del Algoritmo (Firmware)
El control se implementó en un ATmega328P. El siguiente diagrama de flujo describe la rutina de interrupciones y la máquina de estados del algoritmo P&O.

<div align="center">
  <img src="../assets/images/Diagrama.png" alt="Diagrama de Flujo MPPT" width="800"/>
  <p><em>Fig 4. Flujo de ejecución del firmware. Se destaca el uso de la interrupción del Timer 0 para el muestreo del ADC (50ms) y el ajuste del ciclo de trabajo (OCR1A) basado en la perturbación de la potencia.</em></p>
</div>

---

## 4. Diseño de Hardware (PCB)
Para la implementación física, se diseñó un circuito impreso (PCB) que integra la etapa de potencia y el aislamiento de señales.


| Vista 3D | Diseño de Pistas (Layout) |
| :---: | :---: |
| <img src="../assets/images/Captura de pantalla 2025-12-15 123431.png" width="400"/> | <img src="../assets/images/Captura de pantalla 2025-12-15 123324.png" width="400"/> |
| **Fig 5.** Visualización 3D de componentes. | **Fig 6.** Ruteo de pistas en capa Bottom. |



**Notas del diseño:**
* **Driver:** Se observa el uso de un optoacoplador/driver (U1, posible HCPL-3120) para el disparo seguro del MOSFET.
* **Potencia:** Pistas ensanchadas para el manejo de corriente en las líneas de $V_{in}$ y $V_{out}$.
* **Conexiones:** Borneras dedicadas para la entrada del panel solar y la carga.

---

## 5. Resultados Experimentales
Se realizaron pruebas de simulación para validar el comportamiento del algoritmo Perturb & Observe bajo condiciones controladas.

<div align="center">
  <img src="../assets/images/Captura de pantalla 2025-12-15 190709.png" alt="Resultados Simulación P&O" width="800"/>
  <p><em>Fig 7. Respuesta dinámica del sistema. La traza amarilla representa la potencia extraída del panel, la cual converge escalonadamente hacia el punto de máxima potencia (aprox. 25W) en $t \approx 1s$. Una vez alcanzado el MPP, se observa la oscilación característica del algoritmo P&O alrededor del punto óptimo. La traza azul inferior muestra el ajuste correspondiente del ciclo de trabajo <D>.</em></p>
</div>

## 6. Implementación del Controlador PID
Para mejorar la estabilidad y reducir la oscilación en estado estacionario, se implementó un controlador PID. En este esquema, el algoritmo MPPT genera una referencia de voltaje ($V_{ref}$) y el PID ajusta el ciclo de trabajo para que el voltaje del panel ($V_{pv}$) siga dicha referencia.

### 6.1 Esquema de Control PID
<div align="center">
  <img src="../assets/images/Captura de pantalla 2025-12-11 222704.png" alt="Esquematico Simulink PID" width="800"/>
  <p><em>Fig 8. Diagrama de Simulink mostrando el bucle de control PID. El bloque MPPT ahora entrega una referencia de voltaje ($V_{ref}$) que es comparada con el voltaje medido del panel ($V_{pv}$). El error es procesado por el PID para generar la señal PWM.</em></p>
</div>

### 6.2 Resultados con Controlador PID
<div align="center">
  <img src="../assets/images/Captura de pantalla 2025-12-15 191259.png" alt="Resultados Simulación PID" width="800"/>
  <p><em>Fig 9. Desempeño del sistema con controlador PID. Se observa cómo el voltaje del panel (traza amarilla, $V_{PV}$) sigue con precisión a la referencia de voltaje (traza azul, $V_{ref}$) generada por el algoritmo. Como resultado, la potencia extraída (traza naranja) se mantiene estable en el punto máximo (aprox. 25W) con una ondulación considerablemente menor que en el control directo.</em></p>
</div>

> **Conclusión Final:** La inclusión del controlador PID mejora significativamente el desempeño del sistema MPPT, logrando un seguimiento preciso del punto de operación óptimo y reduciendo las pérdidas por oscilación en estado estacionario.